In [ ]:
import pandas as pd
!pip install pyreadstat
import calendar
import os

In [ ]:
path = r'Git\PG\pg-dt-portal-for-nz\data\out-for-datatile'
files = os.listdir(path)
features_group = pd.read_excel(r'Git\PG\pg-dt-portal-for-nz\feature_groups.xlsx')

In [42]:
def get_df_in(path, file):
    cat_dict = {'br_tot.parquet':'MALE B&R', 'deterg_tot.parquet': 'Laundry Detergents','diapers_tot.parquet': 'Diapers','femcare_tot.parquet': 'Feminine Care', 
                'haircond_tot.parquet': 'Hair Conditioners', 'shampoo_tot.parquet':'Shampoos' }
    time_period_type_dict ={ 24 : '2MAT/ 104 we', 12:'MAT/ 52 we', 3: '3MMT/ 12we' }
    
    df_in = pd.read_parquet(f'{path}\\{file}')
    df_in['category'] = [cat_dict[file]]*len( df_in)
    col_to_int = ['duration', 'year', 'month']
    df_in.columns = [x.lower().replace(' ','_') for x in df_in.columns]
    for col in col_to_int:
        df_in[col] = df_in[col].astype(int)
    
    df_in['time_period_type'] = df_in['duration'].map(time_period_type_dict) 
    df_in['period_lbl'] = df_in[['time_period_type', 'year', 'month']].apply(
        lambda row:f" {row['time_period_type'].split('/')[0]}: {calendar.month_abbr[row['month']]} {row['year']}", axis=1 )  
    df_in = df_in.rename({'category_name':'products' }, axis=1 ) 

    df_in = df_in.merge(features_group, left_on=['category', 'products'], right_on =['Category Name', 'Product Name']) 
    
    
    return df_in
    

In [43]:
df_in_one = get_df_in(path, files[0])

In [44]:
df_in_one.head()

,segment,products,position_name_shop,value,volume_gl,volume_su,volume_pack,trips_raw,trips,buyers,...,category,time_period_type,period_lbl,Category Name,Vendor Product ID,Product Name,feature,category_code,product_code,full_label
0,Total Segments,MALE B&R,RUSSIA NATIONAL,3.026921e+07,517066.845114,21976.479036,86035.413342,14784.0,72159.595178,26596.302007,...,MALE B&R,2MAT/ 104 we,2MAT: Jun 2023,MALE B&R,265713,MALE B&R,category,2,2000,MALE B&R
1,Total Segments,P&G MB&R,RUSSIA NATIONAL,2.358849e+07,218803.063896,8958.705337,45080.842432,7670.0,39442.289323,17169.748009,...,MALE B&R,2MAT/ 104 we,2MAT: Jun 2023,MALE B&R,313500,P&G MB&R,producer,2,2001,P&G MB&R
2,Total Segments,Gillette MB&R,RUSSIA NATIONAL,2.357925e+07,218517.915169,8923.709753,45023.069845,7660.0,39403.266562,17163.603520,...,MALE B&R,2MAT/ 104 we,2MAT: Jun 2023,MALE B&R,313501,Gillette MB&R,brand,2,2002,Gillette MB&R
3,Total Segments,P&G Cl.Shv ex KCG SG Bdy Disp,RUSSIA NATIONAL,1.890672e+07,80028.027037,4827.654680,20970.992500,3765.0,18564.414706,10082.422000,...,MALE B&R,2MAT/ 104 we,2MAT: Jun 2023,MALE B&R,499114,P&G Cl.Shv ex KCG SG Bdy Disp,brand,2,2003,P&G Cl.Shv ex KCG SG Bdy Disp
4,Total Segments,BIC MB&R,RUSSIA NATIONAL,2.042912e+06,49703.925821,2062.490023,10312.065378,1928.0,8761.459245,4934.149377,...,MALE B&R,2MAT/ 104 we,2MAT: Jun 2023,MALE B&R,313507,BIC MB&R,producer,2,2004,BIC MB&R


In [ ]:
col_ord = {}
for col in df_in_new:
    if df_in_new[col].dtype =='object':
        col_ord[col] = 0d
    else:
        col_ord[col] = 1
new_ord = sorted(df_in_new.columns, key = col_ord.__getitem__ )
df_in_new = df_in_new[new_ord]
    